In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPooling2D, UpSampling2D, Concatenate, Input
import matplotlib.pyplot as plt
from tensorflow.signal import fft2d, ifft2d
from tensorflow.keras.utils import plot_model
import scipy.io
import mat73
import numpy as np
data_dict=mat73.loadmat("D:/Supriyo/onedrive/OneDrive - iitkgp.ac.in/Sanjay_unet/vi.mat")
loc=np.load("D:/Supriyo/onedrive/OneDrive - iitkgp.ac.in/Sanjay_unet/locations.npy")
print("Data loading completed")
data=data_dict['vi']
x_temp=[]
print("segregation inti......")
for i in range(len(loc)):
    
    temp=data[:,loc[i][0]:loc[i][1]+1]
    sh=np.shape(temp)
    i,j=0,0
    while i<=sh[0] and j<=sh[1]:
        x_temp.append(temp[i:i+128,j:j+128])
        i+=128
        j+=128

Data loading completed
segregation inti......


In [2]:
x_t=[]
for i in x_temp:
    if np.shape(i)==(128,128):
        x_t.append(i)
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
scaler=MinMaxScaler()
data=[]
for i in x_t:
    data.append(scaler.fit_transform(i))

X_train,X_test,y_train,y_test=train_test_split(data,data,test_size=0.1,random_state=42)

In [3]:
np.shape(x_t)

(3312, 128, 128)

In [4]:
del data_dict
del loc
del data
del x_temp
del temp
del y_train
del y_test

In [7]:
inputs = tf.keras.layers.Input((128,128,1))

In [ ]:
# RA-Res-Unet for seismic inversion

def batchnorm_relu(inputs):
    """ Batch Normalization & ReLU """
    x = BatchNormalization()(inputs)
    x = Activation("relu")(x)
    return x

def residual_block(inputs, num_filters, strides=1):
    """ Convolutional Layers """
    x = batchnorm_relu(inputs)
    x = Conv2D(num_filters, 3, padding="same", strides=strides)(x)
    x = batchnorm_relu(x)
    x = Conv2D(num_filters, 3, padding="same", strides=1)(x)
    """ Shortcut Connection (Identity Mapping) """
    s = Conv2D(num_filters, 1, padding="same", strides=strides)(inputs)
    """ Addition """
    x = tf.keras.layers.add([x, s])
    return x

def attention_block(x, filters):
    """ Residual Attention Block """
    g = Conv2D(filters, (1, 1), strides=(1, 1), padding='same')(x)
    theta_x = Conv2D(filters, (1, 1), strides=(1, 1), padding='same')(x)
    phi_g = Conv2D(filters, (1, 1), strides=(1, 1), padding='same')(g)

    theta_x = tf.keras.layers.Permute((3, 1, 2))(theta_x)
    theta_x = tf.keras.layers.Reshape((-1, theta_x.shape[2] * theta_x.shape[3]))(theta_x)
    phi_g = tf.keras.layers.Reshape((-1, phi_g.shape[2] * phi_g.shape[3]))(phi_g)  # Adjust shape here
    f = tf.keras.layers.Dense(phi_g.shape[-1])(theta_x)
    g = tf.keras.layers.Dense(phi_g.shape[-1])(phi_g)
    h = tf.keras.layers.Add()([f, g])
    h = tf.keras.layers.Activation('relu')(h)
    psi_f = tf.keras.layers.Dense(theta_x.shape[-1])(h)
    psi_f = tf.keras.layers.Activation('softmax')(psi_f)
    psi_f = tf.keras.layers.Reshape((x.shape[1], x.shape[2], x.shape[3]))(psi_f)
    x_atten = tf.keras.layers.Multiply()([x, psi_f])
    return x_atten



def decoder_block(inputs, skip_features, num_filters):
    """ Decoder Block """
    x = UpSampling2D((2, 2))(inputs) 
    x = Concatenate()([x, skip_features]) 
    x = residual_block(x, num_filters, strides=1)
    x = attention_block(x, num_filters)  # Add attention after each decoder block
    return x

def build_ra_resunet(input_shape):
    """ RA-RESUNET Architecture """
    inputs = Input(input_shape)

    """ Encoder 1 """ 
    x = Conv2D(64, 3, padding="same", strides=1)(inputs) 
    x = batchnorm_relu(x) 
    x = Conv2D(64, 3, padding="same", strides=1)(x) 
    s = Conv2D(64, 1, padding="same")(inputs) 
    s1 = tf.keras.layers.add([x, s])

    """ Encoder 2, 3 """
    s2 = residual_block(s1, 128, strides=2)
    s3 = residual_block(s2, 256, strides=2)

    """ Bridge """
    b = residual_block(s3, 512, strides=2)

    """ Decoder 1, 2, 3 """ 
    x = decoder_block(b, s3, 256) 
    x = decoder_block(x, s2, 128) 
    x = decoder_block(x, s1, 64)

    """ Classifier """
    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(x)

    """ Model """
    model = tf.keras.Model(inputs, outputs, name="RA-RESUNET")

    return model

if __name__ == "__main__":
    shape = (128, 128, 1)
    model = build_ra_resunet(shape)
    model.summary()

In [12]:
def batchnorm_relu(inputs):
    """ Batch Normalization & ReLU """
    x = BatchNormalization()(inputs)
    x = Activation("relu")(x)
    return x

def residual_block(inputs, num_filters, strides=1):
    """ Convolutional Layers """
    x = batchnorm_relu(inputs)
    x = Conv2D(num_filters, 3, padding="same", strides=strides)(x)
    x = batchnorm_relu(x)
    x = Conv2D(num_filters, 3, padding="same", strides=1)(x)
    """ Shortcut Connection (Identity Mapping) """
    s = Conv2D(num_filters, 1, padding="same", strides=strides)(inputs)
    """ Addition """
    x = tf.keras.layers.add([x, s])
    return x

def attention_block(x, filters):
    """ Residual Attention Block """
    g = Conv2D(filters, (1, 1), strides=(1, 1), padding='same')(x)
    theta_x = Conv2D(filters, (1, 1), strides=(1, 1), padding='same')(x)
    phi_g = Conv2D(filters, (1, 1), strides=(1, 1), padding='same')(g)

    theta_x = tf.keras.layers.Reshape((-1, theta_x.shape[3]))(theta_x)
    phi_g = tf.keras.layers.Reshape((-1, phi_g.shape[3]))(phi_g)

    f = tf.keras.layers.Dense(phi_g.shape[-1])(theta_x)
    g = tf.keras.layers.Dense(phi_g.shape[-1])(phi_g)

    h = tf.keras.layers.Add()([f, g])
    h = tf.keras.layers.Activation('relu')(h)

    psi_f = tf.keras.layers.Dense(theta_x.shape[-1])(h)
    psi_f = tf.keras.layers.Activation('softmax')(psi_f)

    psi_f = tf.keras.layers.Reshape((x.shape[1], x.shape[2], x.shape[3]))(psi_f)
    x_atten = tf.keras.layers.Multiply()([x, psi_f])
    return x_atten


def decoder_block(inputs, skip_features, num_filters):
    """ Decoder Block """
    x = UpSampling2D((2, 2))(inputs) 
    x = Concatenate()([x, skip_features]) 
    x = residual_block(x, num_filters, strides=1)
    x = attention_block(x, num_filters)  # Add attention after each decoder block
    return x

def build_ra_resunet(input_shape):
    """ RA-RESUNET Architecture """
    inputs = Input(input_shape)

    """ Encoder 1 """ 
    x = Conv2D(64, 3, padding="same", strides=1)(inputs) 
    x = batchnorm_relu(x) 
    x = Conv2D(64, 3, padding="same", strides=1)(x) 
    s = Conv2D(64, 1, padding="same")(inputs) 
    s1 = tf.keras.layers.add([x, s])

    """ Encoder 2, 3 """
    s2 = residual_block(s1, 128, strides=2)
    s3 = residual_block(s2, 256, strides=2)

    """ Bridge """
    b = residual_block(s3, 512, strides=2)

    """ Decoder 1, 2, 3 """ 
    x = decoder_block(b, s3, 256) 
    x = decoder_block(x, s2, 128) 
    x = decoder_block(x, s1, 64)

    """ Classifier """
    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(x)

    """ Model """
    model = tf.keras.Model(inputs, outputs, name="RA-RESUNET")

    return model

if __name__ == "__main__":
    shape = (128, 128, 1)
    model = build_ra_resunet(shape)
    model.summary()

Model: "RA-RESUNET"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_72 (Conv2D)             (None, 128, 128, 64  640         ['input_6[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_36 (BatchN  (None, 128, 128, 64  256        ['conv2d_72[0][0]']              
 ormalization)                  )                                                        

In [5]:
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping

In [30]:

from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping


# RA-Res-Unet for seismic inversion

def batchnorm_relu(inputs):
    """ Batch Normalization & ReLU """
    x = BatchNormalization()(inputs)
    x = Activation("relu")(x)
    return x

def residual_block(inputs, num_filters, strides=1, kernel_regularizer=None):
    """ Convolutional Layers with Kernel Regularizer """
    x = batchnorm_relu(inputs)
    x = Conv2D(num_filters, 3, padding="same", strides=strides, kernel_regularizer=kernel_regularizer)(x)
    x = batchnorm_relu(x)
    x = Conv2D(num_filters, 3, padding="same", strides=1, kernel_regularizer=kernel_regularizer)(x)
    """ Shortcut Connection (Identity Mapping) """
    s = Conv2D(num_filters, 1, padding="same", strides=strides)(inputs)
    """ Addition """
    x = tf.keras.layers.add([x, s])
    return x

def attention_block(x, filters, kernel_regularizer=None):
    """ Residual Attention Block """
    g = Conv2D(filters, (1, 1), strides=(1, 1), padding='same')(x)
    theta_x = Conv2D(filters, (1, 1), strides=(1, 1), padding='same')(x)
    phi_g = Conv2D(filters, (1, 1), strides=(1, 1), padding='same')(g)

    theta_x = tf.keras.layers.Reshape((-1, theta_x.shape[3]))(theta_x)
    phi_g = tf.keras.layers.Reshape((-1, phi_g.shape[3]))(phi_g)

    f = tf.keras.layers.Dense(phi_g.shape[-1], kernel_regularizer=kernel_regularizer)(theta_x)
    g = tf.keras.layers.Dense(phi_g.shape[-1], kernel_regularizer=kernel_regularizer)(phi_g)

    h = tf.keras.layers.Add()([f, g])
    h = tf.keras.layers.Activation('relu')(h)

    psi_f = tf.keras.layers.Dense(theta_x.shape[-1], kernel_regularizer=kernel_regularizer)(h)
    psi_f = tf.keras.layers.Activation('softmax')(psi_f)

    psi_f = tf.keras.layers.Reshape((x.shape[1], x.shape[2], x.shape[3]))(psi_f)
    x_atten = tf.keras.layers.Multiply()([x, psi_f])
    return x_atten

def decoder_block(inputs, skip_features, num_filters, kernel_regularizer=None):
    """ Decoder Block """
    x = UpSampling2D((2, 2))(inputs) 
    x = Concatenate()([x, skip_features]) 
    x = residual_block(x, num_filters, strides=1, kernel_regularizer=kernel_regularizer)
    x = attention_block(x, num_filters, kernel_regularizer=kernel_regularizer)  # Add attention after each decoder block
    return x

def build_ra_resunet(input_shape, kernel_regularizer=None):
    """ RA-RESUNET Architecture """
    inputs = Input(input_shape)

    """ Encoder 1 """ 
    x = Conv2D(32, 3, padding="same", strides=1, kernel_regularizer=kernel_regularizer)(inputs) 
    x = batchnorm_relu(x) 
    x = Conv2D(32, 3, padding="same", strides=1, kernel_regularizer=kernel_regularizer)(x) 
    s = Conv2D(32, 1, padding="same")(inputs) 
    s1 = tf.keras.layers.add([x, s])

    """ Encoder 2, 3 """
    s2 = residual_block(s1, 64, strides=2, kernel_regularizer=kernel_regularizer)
    s3 = residual_block(s2, 128, strides=2, kernel_regularizer=kernel_regularizer)

    """ Bridge """
    b = residual_block(s3, 256, strides=2, kernel_regularizer=kernel_regularizer)

    """ Decoder 1, 2, 3 """ 
    x = decoder_block(b, s3, 128, kernel_regularizer=kernel_regularizer) 
    x = decoder_block(x, s2, 64, kernel_regularizer=kernel_regularizer) 
    x = decoder_block(x, s1, 32, kernel_regularizer=kernel_regularizer)

    """ Classifier """
    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(x)

    """ Model """
    model = tf.keras.Model(inputs, outputs, name="RA-RESUNET")

    return model

if __name__ == "__main__":
    shape = (128, 128, 1)
    model = build_ra_resunet(shape, kernel_regularizer=l2(1e-4))
    model.summary()

   

Model: "RA-RESUNET"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_93 (Conv2D)             (None, 128, 128, 32  320         ['input_5[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_39 (BatchN  (None, 128, 128, 32  128        ['conv2d_93[0][0]']              
 ormalization)                  )                                                        

In [31]:

from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping


# RA-Res-Unet for seismic inversion

def batchnorm_relu(inputs):
    """ Batch Normalization & ReLU """
    x = BatchNormalization()(inputs)
    x = Activation("relu")(x)
    return x

def residual_block(inputs, num_filters, strides=1, kernel_regularizer=None):
    """ Convolutional Layers with Kernel Regularizer """
    x = batchnorm_relu(inputs)
    x = Conv2D(num_filters, 3, padding="same", strides=strides, kernel_regularizer=kernel_regularizer)(x)
    x = batchnorm_relu(x)
    x = Conv2D(num_filters, 3, padding="same", strides=1, kernel_regularizer=kernel_regularizer)(x)
    """ Shortcut Connection (Identity Mapping) """
    s = Conv2D(num_filters, 1, padding="same", strides=strides)(inputs)
    """ Addition """
    x = tf.keras.layers.add([x, s])
    return x

def attention_block(x, filters, kernel_regularizer=None):
    """ Residual Attention Block """
    g = Conv2D(filters, (1, 1), strides=(1, 1), padding='same')(x)
    theta_x = Conv2D(filters, (1, 1), strides=(1, 1), padding='same')(x)
    phi_g = Conv2D(filters, (1, 1), strides=(1, 1), padding='same')(g)

    theta_x = tf.keras.layers.Reshape((-1, theta_x.shape[3]))(theta_x)
    phi_g = tf.keras.layers.Reshape((-1, phi_g.shape[3]))(phi_g)

    f = tf.keras.layers.Dense(phi_g.shape[-1], kernel_regularizer=kernel_regularizer)(theta_x)
    g = tf.keras.layers.Dense(phi_g.shape[-1], kernel_regularizer=kernel_regularizer)(phi_g)

    h = tf.keras.layers.Add()([f, g])
    h = tf.keras.layers.Activation('relu')(h)

    psi_f = tf.keras.layers.Dense(theta_x.shape[-1], kernel_regularizer=kernel_regularizer)(h)
    psi_f = tf.keras.layers.Activation('softmax')(psi_f)

    psi_f = tf.keras.layers.Reshape((x.shape[1], x.shape[2], x.shape[3]))(psi_f)
    x_atten = tf.keras.layers.Multiply()([x, psi_f])
    return x_atten

def decoder_block(inputs, skip_features, num_filters, kernel_regularizer=None):
    """ Decoder Block """
    x = UpSampling2D((2, 2))(inputs) 
    x = Concatenate()([x, skip_features]) 
    x = residual_block(x, num_filters, strides=1, kernel_regularizer=kernel_regularizer)
    x = attention_block(x, num_filters, kernel_regularizer=kernel_regularizer)  # Add attention after each decoder block
    return x

def build_ra_resunet(input_shape, kernel_regularizer=None):
    """ RA-RESUNET Architecture """
    inputs = Input(input_shape)

    """ Encoder 1 """ 
    x = Conv2D(32, 3, padding="same", strides=1, kernel_regularizer=kernel_regularizer)(inputs) 
    x = batchnorm_relu(x) 
    x = Conv2D(32, 3, padding="same", strides=1, kernel_regularizer=kernel_regularizer)(x) 
    s = Conv2D(32, 1, padding="same")(inputs) 
    s1 = tf.keras.layers.add([x, s])

    """ Encoder 2, 3 """
    s2 = residual_block(s1, 64, strides=2, kernel_regularizer=kernel_regularizer)
    s3 = residual_block(s2, 128, strides=2, kernel_regularizer=kernel_regularizer)

    """ Bridge """
    b = residual_block(s3, 256, strides=2, kernel_regularizer=kernel_regularizer)

    """ Decoder 1, 2, 3 """ 
    x = decoder_block(b, s3, 128, kernel_regularizer=kernel_regularizer) 
    x = decoder_block(x, s2, 64, kernel_regularizer=kernel_regularizer) 
    x = decoder_block(x, s1, 32, kernel_regularizer=kernel_regularizer)

    """ Classifier """
    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(x)

    """ Model """
    model = tf.keras.Model(inputs, outputs, name="RA-RESUNET")

    return model

if __name__ == "__main__":
    shape = (128, 128, 1)
    model = build_ra_resunet(shape, kernel_regularizer=l2(1e-4))
    model.summary()

   

Model: "RA-RESUNET"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_124 (Conv2D)            (None, 128, 128, 32  320         ['input_6[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_52 (BatchN  (None, 128, 128, 32  128        ['conv2d_124[0][0]']             
 ormalization)                  )                                                        

In [35]:
from tensorflow.keras.models import load_model
import visualkeras

# Load your pre-trained model
model = load_model('D:/Supriyo/onedrive/OneDrive - iitkgp.ac.in/Thesis Concrete Work/UNET-FFT/RA_Res_Unet.h5')

# Visualize the model
visualkeras.layered_view(model, to_file='model_visualization.png').show()


TypeError: 'int' object is not iterable

In [34]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_visualization.png', show_shapes=True, show_layer_names=True)


You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [20]:
 # Compile the model with an optimizer and loss function
    model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

    # Add Early Stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Train the model
    model.fit(X_train, y_train, batch_size=32, epochs=50, validation_split=0.1, callbacks=[early_stopping])

IndentationError: unexpected indent (726242376.py, line 2)

In [8]:
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
model.fit(tf.expand_dims(X_train, axis=-1),tf.expand_dims(X_train, axis=-1), batch_size=32, epochs=10, validation_split=0.1)

Epoch 1/10
84/84 [==============================] - 710s 8s/step - loss: 0.0837 - accuracy: 0.0167 - val_loss: 0.0785 - val_accuracy: 0.0162
Epoch 2/10
84/84 [==============================] - 827s 10s/step - loss: 0.0182 - accuracy: 0.0169 - val_loss: 0.0577 - val_accuracy: 0.0165
Epoch 3/10
84/84 [==============================] - 786s 9s/step - loss: 0.0075 - accuracy: 0.0169 - val_loss: 0.0482 - val_accuracy: 0.0165
Epoch 4/10
84/84 [==============================] - 3813s 46s/step - loss: 0.0039 - accuracy: 0.0169 - val_loss: 0.0389 - val_accuracy: 0.0165
Epoch 5/10
84/84 [==============================] - 788s 9s/step - loss: 0.0024 - accuracy: 0.0169 - val_loss: 0.0264 - val_accuracy: 0.0165
Epoch 6/10
84/84 [==============================] - 744s 9s/step - loss: 0.0016 - accuracy: 0.0169 - val_loss: 0.0169 - val_accuracy: 0.0165
Epoch 7/10
84/84 [==============================] - 753s 9s/step - loss: 0.0012 - accuracy: 0.0169 - val_loss: 0.0091 - val_accuracy: 0.0165
Epoch 8/10

In [9]:
model.save('RA_Res_Unet.h5')

In [28]:
from tensorflow.keras.models import load_model
# Load your pre-trained model
import visualkeras
#model=load_model('D:/Supriyo/onedrive/OneDrive - iitkgp.ac.in/Thesis Concrete Work/UNET-FFT/Unet_fft_fft.h5')
visualkeras.layered_view(model).show()


TypeError: 'int' object is not iterable

In [12]:
import visualkeras
from tensorflow.keras.models import load_model

# Load your pre-trained model
model = load_model('RA_Res_Unet.h5')

# Visualize the model and display using the system viewer
visualkeras.layered_view(model).show()

# Write the visualization to a file
visualkeras.layered_view(model, to_file='output.png')

# Write to file and show using the system viewer
visualkeras.layered_view(model, to_file='output.png').show()

# The last line will also display the visualization in your Jupyter Notebook if you're using one.


TypeError: 'int' object is not iterable

In [14]:
!pip install pydot


[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
!pip install graphviz

                                              0.0/47.0 kB ? eta -:--:--
     ----------------                       20.5/47.0 kB 682.7 kB/s eta 0:00:01
     ------------------------               30.7/47.0 kB 330.3 kB/s eta 0:00:01
     ---------------------------------      41.0/47.0 kB 393.8 kB/s eta 0:00:01
     ---------------------------------      41.0/47.0 kB 393.8 kB/s eta 0:00:01
     ---------------------------------      41.0/47.0 kB 393.8 kB/s eta 0:00:01
     -------------------------------------- 47.0/47.0 kB 157.3 kB/s eta 0:00:00



[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip
